In [51]:
import pandas as pd 
import numpy as np 
import plotly.express as px 
import petpy 
import pickle
import sqlite3
import time
import datetime

In [37]:
with open('/home/malcolm/credentials/petfinder_credentials.pkl', 'rb') as hnd:
    creds = pickle.load(hnd)

## Create API Request yourself

In [38]:
import urllib
import requests

In [39]:
endpoint = 'oauth2/token'
host = 'http://api.petfinder.com/v2/'
url = urllib.parse.urljoin(host, endpoint)

data = {
    'grant_type': 'client_credentials',
    'client_id': creds['key'],
    'client_secret': creds['secret']
}

r = requests.post(url, data=data)
api_key = r.json()['access_token']

In [40]:
animals_endpt = 'https://api.petfinder.com/v2/animals/'
params = dict(type='dog', 
    location='10475', distance=30,
    limit=100, page=1
             )
headers = {'Authorization' : 'Bearer ' + api_key}
# headers = {'Authorization' : 'Bearer ' + pf_key}

In [41]:
results = requests.get(animals_endpt
                       , params=params
                       , headers=headers)
results

<Response [200]>

In [44]:
dogs_rslts_json = results.json()
dogs_rslts_json.keys()

dict_keys(['animals', 'pagination'])

2303

In [45]:
n_dog_pages = dogs_rslts_json['pagination']['total_pages']
n_dog_pages

24

In [46]:
params['type'] = 'cat'
results = requests.get(animals_endpt
                       , params=params
                       , headers=headers)
assert(results.status_code == 200), f"Status code not 200: It's {results.status_code}"
cats_rslts_json = results.json()
n_cat_pages = cats_rslts_json['pagination']['total_pages']
n_cat_pages

37

## Get All Animals For Region


In [47]:
pf = petpy.Petfinder(key=creds['key'], secret=creds['secret'])

In [48]:
params.keys()

dict_keys(['type', 'location', 'distance', 'limit', 'page'])

In [49]:
petpy_dict = params.copy()
# petpy_dict['animal_type'] = petpy_dict.pop('type')
petpy_dict['animal_type'] = 'dog'
petpy_dict['results_per_page'] = petpy_dict.pop('limit')
petpy_dict['pages'] = n_dog_pages
petpy_dict['return_df'] = True
del(petpy_dict['page'])
if 'type' in petpy_dict.keys():
    del(petpy_dict['type'])
print(petpy_dict)

{'location': '10475', 'distance': 30, 'animal_type': 'dog', 'results_per_page': 100, 'pages': 24, 'return_df': True}


In [53]:
adoptable_dogs = pf.animals(**petpy_dict)
adoptable_dogs['date_saved'] = str(datetime.datetime.now().date())
adoptable_dogs

,id,organization_id,url,type,species,age,gender,size,coat,tags,...,contact.address.postcode,contact.address.country,animal_id,animal_type,organization_id,primary_photo_cropped.small,primary_photo_cropped.medium,primary_photo_cropped.large,primary_photo_cropped.full,date_saved
0,52223615,NY1506,https://www.petfinder.com/dog/scruffy-nyc-5222...,Dog,Dog,Adult,Male,Large,None,[],...,10013,US,52223615,dog,ny1506,NaN,NaN,NaN,NaN,2021-07-03
1,52223614,NY1506,https://www.petfinder.com/dog/sena-atl-5222361...,Dog,Dog,Baby,Female,Small,None,[],...,10013,US,52223614,dog,ny1506,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,2021-07-03
2,52223145,NY615,https://www.petfinder.com/dog/jenny-52223145/n...,Dog,Dog,Adult,Female,Medium,Short,[],...,11561,US,52223145,dog,ny615,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,2021-07-03
3,52223012,NJ516,https://www.petfinder.com/dog/potato-chip-5222...,Dog,Dog,Young,Female,Medium,Short,[],...,07470,US,52223012,dog,nj516,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,2021-07-03
4,52222683,NY1374,https://www.petfinder.com/dog/lobby-52222683/n...,Dog,Dog,Adult,Female,Small,Short,[],...,11211,US,52222683,dog,ny1374,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,2021-07-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2298,27249421,NY102,https://www.petfinder.com/dog/luke-27249421/ny...,Dog,Dog,Adult,Male,Medium,Short,"[playful, goofy, charming]",...,11364,US,27249421,dog,ny102,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,2021-07-03
2299,25874439,NY892,https://www.petfinder.com/dog/ibizan-hounds-na...,Dog,Dog,Adult,Male,Large,Short,[],...,10032,US,25874439,dog,ny892,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,2021-07-03
2300,25777517,NY977,https://www.petfinder.com/dog/italian-greyhoun...,Dog,Dog,Adult,Male,Small,None,[],...,10520,US,25777517,dog,ny977,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,2021-07-03
2301,25009612,NY955,https://www.petfinder.com/dog/franklin-2500961...,Dog,Dog,Senior,Male,Medium,Short,"[beautiful, smart, funny, kind, dog friendly, ...",...,11542,US,25009612,dog,ny955,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,2021-07-03


In [55]:
time.sleep(5)
petpy_dict['animal_type'] = 'cat'
petpy_dict['pages'] = n_cat_pages
adoptable_cats = pf.animals(**petpy_dict)
adoptable_cats['date_saved'] = str(datetime.datetime.now().date())
adoptable_cats

,id,organization_id,url,type,species,age,gender,size,coat,tags,...,contact.address.address2,contact.address.city,contact.address.state,contact.address.postcode,contact.address.country,animal_id,animal_type,organization_id,primary_photo_cropped,date_saved
0,52223585,NY1379,https://www.petfinder.com/cat/molly-52223585/n...,Cat,Cat,Young,Female,Medium,None,[],...,None,Woodmere,NY,11598,US,52223585,cat,ny1379,NaN,2021-07-03
1,52223539,NY12,https://www.petfinder.com/cat/mike-52223539/ny...,Cat,Cat,Baby,Male,Medium,None,[],...,None,New York,NY,10029,US,52223539,cat,ny12,NaN,2021-07-03
2,52223532,NY08,https://www.petfinder.com/cat/stella-52223532/...,Cat,Cat,Adult,Female,Medium,None,[],...,None,Brooklyn,NY,11208,US,52223532,cat,ny08,NaN,2021-07-03
3,52223531,NY08,https://www.petfinder.com/cat/titi-52223531/ny...,Cat,Cat,Senior,Female,Medium,None,[],...,None,Brooklyn,NY,11208,US,52223531,cat,ny08,NaN,2021-07-03
4,52223438,NY479,https://www.petfinder.com/cat/lightning-522234...,Cat,Cat,Baby,Male,Small,None,[],...,Between 1st Avenue & FDR Drive,New York,NY,10016,US,52223438,cat,ny479,NaN,2021-07-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3615,9619546,NY339,https://www.petfinder.com/cat/spunky-9619546/n...,Cat,Cat,Senior,Female,Small,None,[],...,None,New York,NY,10022,US,9619546,cat,ny339,NaN,2021-07-03
3616,8585916,NJ29,https://www.petfinder.com/cat/foster-program-8...,Cat,Cat,Baby,Female,Medium,None,[],...,None,Teterboro,NJ,07608,US,8585916,cat,nj29,NaN,2021-07-03
3617,7383586,NY04,https://www.petfinder.com/cat/brie-good-news-7...,Cat,Cat,Adult,Female,Large,Short,[],...,None,Huntington,NY,11743,US,7383586,cat,ny04,NaN,2021-07-03
3618,6744371,CT13,https://www.petfinder.com/cat/kittens-6744371/...,Cat,Cat,Baby,Male,Medium,None,[],...,None,New Canaan,CT,06840,US,6744371,cat,ct13,NaN,2021-07-03


## Export to SQL

In [97]:
# Prep dataset for export into SQL 
adoptable_dogs = adoptable_dogs.loc[:,~adoptable_dogs.columns.duplicated()]
adoptable_cats = adoptable_cats.loc[:,~adoptable_cats.columns.duplicated()]
for x in ['tags', 'photos', 'videos']:
    adoptable_dogs[x] = str(adoptable_dogs[x])
    adoptable_cats[x] = str(adoptable_cats[x])

/home/malcolm/main/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/home/malcolm/main/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [99]:
con = sqlite3.connect('/home/malcolm/petfinder/data/petfinder.db')
cursor = con.cursor()

In [64]:
table_name_suffix = f"{petpy_dict['location']}_{petpy_dict['distance']}mi"
table_name_suffix



'10475_30mi'

In [106]:
adoptable_dogs.to_sql('dogs_'+table_name_suffix, con, index=False, if_exists='append')
adoptable_cats.to_sql('cats_'+table_name_suffix, con, index=False, if_exists='append')

## Graveyard 

In [ ]:
pf = petpy.Petfinder(key=creds['key'], secret=creds['secret'])

In [ ]:
# temp_dogs = pf.animals(animal_type='dog', 
#     location=10475, distance=50,
#     results_per_page=100, pages=1, return_df=False)
temp_dogs.keys()

In [ ]:
adoptable_dogs = pf.animals(animal_type='dog', 
    location=10475, distance=25,
    results_per_page=100, pages=2, return_df=True)
adoptable_dogs

In [ ]:
adoptable_dogs.columns

In [ ]:
vc = adoptable_dogs['contact.address.city'].value_counts()
print(vc.shape)
vc.head(10)

In [ ]:
vc.sum()

In [ ]:
adoptable_dogs['status'].value_counts()

In [110]:
con.commit()
con.close()